# Steps to Preprocess the CORDEX-Australasia Ensemble for Input into Climpact
The code below includes all relevant preprocessing steps taken to prepare the daily tmin, tmax, and precip CORDEX-Australasia data for input into Climpact. <br>
Note, the following steps are taken to prepare the daily files:
- Merge the daily files into one file (stored as 1 file per year)
- Interpolate the necessary files to a consistent 0.5 X 0.5 deg, rectilinear grid (noted as AUS-44i in the file naming)
- Create a new catalogue with the necessary metadata to use with the cordex-climpact software (see below)

Using Climpact, we then calculate the full set of Climpact climate indices at a common 0.5 x 0.5 degree spatial resolution for the historical, RCP4.5, and RCP8.5 historical simulations. More information about climpact can be found at: https://climpact-sci.org/

The revised Climpact software used to batch create the CORDEX-Australasia indices can be found at: https://github.com/coecms/cordex-climpact

Note, these preprocessing steps have already been done for the daily, historical and RCP8.5 precipitation datasets. <br>
Authors: Rachael N. Isphording, Josh Amoils, and (Alex) Ying Lung Liu

In [3]:
# Import Modules
import os
import fnmatch
import pandas as pd

## Step 1: Find the files that we want and concatenate daily files into one dataset
Note, this has already been done for the historical and RCP8.5 precip, but needs to be done for the RCP4.5 simulations

In [2]:
# List regions (i.e. domains) that we are interested in
domains = ["AUS-22/", "AUS-44/", "AUS-44i/"]

# Define master paths (below are the two master paths on Gadi for the original CORDEX output
master_path = '/g/data/al33/replicas/cordex/output/'
#master_path = '/g/data/rr3/publications/CORDEX/output/'

output_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/'

In [3]:
# Define useful functions

# Get dates from the first and last file to add to the new, concatenated (merged) file name
def extract_dates_from_filenames(filenames):
    start_date = None
    end_date = None

    for filename in filenames:
        date_range = filename.split('_')[-1].split('.')[0]
        range_start, range_end = date_range.split('-')

        if start_date is None or range_start < start_date:
            start_date = range_start
        if end_date is None or range_end > end_date:
            end_date = range_end

    return start_date, end_date

# Merge files using CDO
def merge_files(input_file, output_file):
    
    print("!START:\n input files: " + input_file + "\n output file: " + output_file)
    
    !cdo mergetime {input_file} {output_file}

    return(print("!COMPLETED:\n input files: " + input_file + "\n output file: " + output_file))

# Recursive function to parse directories and process files
def parse_directory(directory, domain):
    for root, dirs, files in os.walk(os.path.join(directory, domain)):
        if files:
            # Filter files based on keywords; the '_' after the variable name ensures that we don't grab the bias corrected or filtered datasets
            files = [file for file in files if ('r1i1p1' in file and 'day' in file and ('tasmin_' in file or 'tasmax_' in file or 'pr_' in file))]
            if not files:
                continue
            # Extract start and end dates from filenames
            start_date, end_date = extract_dates_from_filenames(files)
            
            # Remove dates from file name
            # Find the length of the substring to remove
            substring_length = len("_19500101-20051230.nc")

            # Remove the substring using string splicing
            new_filename = files[0][:-substring_length] + files[0][:0]
            
            # Generate output file path
            input_file = os.path.join(root, f'{new_filename}_*.nc')
            output_file = os.path.join(output_path, domain, f'{new_filename}_{start_date}-{end_date}.nc')
            
            # Check if file already exists
            if os.path.exists(output_file):
                
                print("File already exists: " + output_file)
            
            else:
                
                # Merge files using CDO command
                merge_files(input_file, output_file)

# Call the function to start parsing the master directory
# DOMAINS in all-caps to prevent accidental running
for domain in DOMAINS:
    parse_directory(master_path, domain)

File already exists: /scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/AUS-22/tasmax_AUS-22_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_CLMcom-HZG-CCLM5-0-15_v1_day_20060101-21001231.nc
File already exists: /scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/AUS-22/pr_AUS-22_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_CLMcom-HZG-CCLM5-0-15_v1_day_20060101-21001231.nc
File already exists: /scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/AUS-22/tasmin_AUS-22_MPI-M-MPI-ESM-LR_rcp85_r1i1p1_CLMcom-HZG-CCLM5-0-15_v1_day_20060101-21001231.nc
File already exists: /scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/AUS-22/tasmax_AUS-22_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-HZG-CCLM5-0-15_v1_day_19490101-20051231.nc
File already exists: /scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/AUS-22/pr_AUS-22_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-HZG-CCLM5-0-15_v1_day_19490101-20051231.nc
File already exists: /scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/AUS-22/tasmin_AUS-22_MPI-M-MPI-ESM-LR_historical_r1i1p1

## Step 2: Interpolate and remap the AUS-22i datasets to AUS-44i (0.5 X 0.5 deg)
Note, bilinear interpolation will be used for temperature data and first order conservative regridding will be used for precipitation data <br>
The 'my_projection_AUS22.txt' file was created prior to running the section below using the CDO command: <br>
cdo griddes tasmin_AUS-22_NCC-NorESM1-M_rcp85_r1i1p1_ICTP-RegCM4-7_v0_day_20060101-20991231.nc > my_projection_AUS22.txt <br>
This steps assumes a consistent grid description for files of the same domain. <br>
Also note, the AUS-44 files had previously been interpolated to the AUS-44i grid.

In [90]:
# List regions (i.e. domains) that we are interested in and time periods for sorting
domains = ["AUS-22"]
time_periods = ["historical", "rcp45", "rcp85"]

# Define input and output directories
input_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step1_concat/'
output_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/'

In [4]:
# Loop through directories; DOMAINS changed to all-caps to prevent accidental running of the script
for domain in DOMAINS:
    
    for time_period in time_periods:
        
        # Iterate through the files in the given directory
        for filename in os.listdir(input_master_path + domain + '/' + time_period):
            
            print("Starting: ", filename)
            
            # Define new filename after regridding and full output path
            new_filename = filename.replace(f"{domain}", "AUS-44i")
            new_full_output_path = os.path.join(output_master_path, time_period, new_filename)
            
            print("New output path : ", new_full_output_path)
            
            # Check if the file already exists
            if os.path.exists(new_full_output_path):
                print("File already exists: " + new_full_output_path)
                
            else:
                
                # Set the gridtype and store the changed gridtype as 'temp.nc'
                !cdo setgrid,{input_master_path + 'my_projection_AUS22.txt'} {input_master_path + domain + '/' + time_period + '/' + filename} {output_master_path + 'temp.nc'}
            
                # Perform the regridding using cdo remapycon (first order conservative regriddingand save in the folder based on the time period
                if filename.startswith('pr_AUS-22'):
                
                    !cdo remapycon,{output_master_path + 'AUS_44i_time1.nc'} {output_master_path + 'temp.nc'} {new_full_output_path}
            
                # perform bilinear interpolation for the temperature files
                elif filename.startswith('tasm'):
                
                    !cdo remapbil,{output_master_path + 'AUS_44i_time1.nc'} {output_master_path + 'temp.nc'} {new_full_output_path}

Starting:  tasmin_AUS-22_MOHC-HadGEM2-ES_historical_r1i1p1_GERICS-REMO2015_v1_day_19700101-20051230.nc
New output path :  /scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/tasmin_AUS-44i_MOHC-HadGEM2-ES_historical_r1i1p1_GERICS-REMO2015_v1_day_19700101-20051230.nc
File already exists: /scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/tasmin_AUS-44i_MOHC-HadGEM2-ES_historical_r1i1p1_GERICS-REMO2015_v1_day_19700101-20051230.nc
Starting:  pr_AUS-22_MPI-M-MPI-ESM-LR_historical_r1i1p1_GERICS-REMO2015_v1_day_19700101-20051231.nc
New output path :  /scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/pr_AUS-44i_MPI-M-MPI-ESM-LR_historical_r1i1p1_GERICS-REMO2015_v1_day_19700101-20051231.nc
File already exists: /scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/pr_AUS-44i_MPI-M-MPI-ESM-LR_historical_r1i1p1_GERICS-REMO2015_v1_day_19700101-20051231.nc
Starting:  pr_AUS-22_MPI-M-MPI-ESM-LR_historical_r1i1p1_CLMcom-HZG-

## Step 3: Create a new intake catalogue with necessary metadata for the interpolated files
This intake catalogue file needs to be updated in the preprocess.py script of the cordex-climpact software. 

In [4]:
# Read original catalogue
old_catalogue = pd.read_csv('/g/data/w40/ri9247/josh/catalogue_AUS44i.csv')

In [5]:
# Print original catalogue for sanity check
old_catalogue

,rcm_name,driving_model,experiment,path,project,product,domain,institute,ensemble,rcm_name_id,rcm_version,time_frequency,variable,version,date_range
0,CCAM-2008,CCCma-CanESM2,historical,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,CSIRO,r1i1p1,CCAM-2008,v1,day,pr,v20201123,19600101-20051231
1,CCAM-2008,CCCma-CanESM2,rcp85,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,CSIRO,r1i1p1,CCAM-2008,v1,day,pr,v20201123,20060101-20991231
2,WRF360J,CCCma-CanESM2,historical,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,UNSW,r1i1p1,UNSW-WRF360J,v1,day,pr,v20210629,19510101-20051231
3,WRF360J,CCCma-CanESM2,rcp85,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,UNSW,r1i1p1,UNSW-WRF360J,v1,day,pr,v20210629,20060101-20991231
4,WRF360K,CCCma-CanESM2,historical,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,UNSW,r1i1p1,UNSW-WRF360K,v1,day,pr,v20210629,19510101-20051231
5,WRF360K,CCCma-CanESM2,rcp85,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,UNSW,r1i1p1,UNSW-WRF360K,v1,day,pr,v20210629,20060101-20991231
6,CCAM-1704,CNRM-CERFACS-CNRM-CM5,historical,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-1704,v1,day,pr,v20201123,19600101-20051231
7,CCAM-1704,CNRM-CERFACS-CNRM-CM5,rcp85,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-1704,v1,day,pr,v20201123,20060101-20991231
8,CCAM-1704,CSIRO-BOM-ACCESS1-0,historical,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-1704,v1,day,pr,v20201123,19600101-20051231
9,CCAM-1704,CSIRO-BOM-ACCESS1-0,rcp85,/g/data/w40/ri9247/CORDEX-Australasia/data/rcm...,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-1704,v1,day,pr,v20201123,20060101-20991231


In [6]:
# Define a new catalogue with the same columns as the original catalogue
new_catalogue = pd.DataFrame(columns = old_catalogue.columns)

In [7]:
# Define constants for new catalogue
project = 'cordex'
product = 'output'

# The version gets dropped in Scott's code, so we'll force a version, choosing the birthday shared by me and Baby Spice. It is also National Hug Day. 
version = 'v19920121'

# Define list with experiments
experiments = [
    'historical',
    'rcp85',
    'rcp45'
]

# Define master path of input datasets
master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/'

Loop through the directory of input data and fill in the rest of the columns in the catalogue. <br>
Note that the files follow the following naming convention: <br>
{variable} _ {domain} _ {driving_model} _ {experiment} _ {ensemble} _ {institue}-{rcm_name} _ {rcm_version} _ {time_frequency} _ {date_range}.nc <br>
{variable} _ {domain} _ {driving_model} _ {experiment} _ {ensemble} _ {rcm_name_id} _ {rcm_version} _ {time_frequency} _ {date_range}.nc <br>
An example: <br>
tasmin _ AUS-44i _ NOAA-GFDL-GFDL-ESM2M _ rcp85 _ r1i1p1 _ CSIRO-CCAM-2008 _ v2 _ day _ 20060101-20991231.nc

In [8]:
# Loop through the time period (aka experiments) directories in our pre-processed files
for experiment in experiments:
    directory = master_path + experiment
    
    # Get the individual files
    for filename in os.listdir(directory):
        
        # Extract the different attributes from the filename
        attributes = filename.split('_')
        
        # Define each attribute in the list (see above file naming structure
        variable = attributes[0]
        domain = attributes[1]
        driving_model = attributes[2]
        # experiment is defined via the looping
        ensemble = attributes[4]
        rcm_name_id = attributes[5]
        
        # A bit of hard-coding because the institute name for 'CLMcom-HZG' has a hyphen in it
        if 'CLMcom-HZG' in rcm_name_id:
            institute = 'CLMcom-HZG'
            rcm_name = 'CCLM5-0-15'
              
        else:
            # split the rcm_name_id at the first hyphen to get the institute and the rcm_name
            rcm_attributes = rcm_name_id.split('-', 1)
            institute = rcm_attributes[0].strip()
            rcm_name = rcm_attributes[1].strip()
            
        rcm_version = attributes[6]
        time_frequency = attributes[7]
        date_range = attributes[8]
        
        # Define the full path
        path = master_path + experiment + '/' + filename
        
        # Make a new dataframe with the file entry
        catalogue_entry = pd.DataFrame({'rcm_name': rcm_name,
                                        'driving_model': driving_model,
                                        'experiment': experiment,
                                        'path': path,
                                        'project': project,
                                        'product': product,
                                        'domain': domain,
                                        'institute': institute,
                                        'ensemble': ensemble,
                                        'rcm_name_id': rcm_name_id,
                                        'rcm_version': rcm_version,
                                        'time_frequency': time_frequency,
                                        'variable': variable,
                                        'version': version,
                                        'date_range': date_range
                                       }, index=[0])
        
        # Add everything to the catalogue
        new_catalogue = pd.concat([new_catalogue, catalogue_entry], ignore_index = True)
        

In [9]:
# Print new_catalogue to check
pd.set_option('display.max_colwidth', None)
new_catalogue

,rcm_name,driving_model,experiment,path,project,product,domain,institute,ensemble,rcm_name_id,rcm_version,time_frequency,variable,version,date_range
0,CCAM-1704,CSIRO-BOM-ACCESS1-0,historical,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/tasmax_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_CSIRO-CCAM-1704_v1_day_19600101-20051231.nc,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-1704,v1,day,tasmax,v19920121,19600101-20051231.nc
1,RegCM4-7,NCC-NorESM1-M,historical,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/tasmin_AUS-44i_NCC-NorESM1-M_historical_r1i1p1_ICTP-RegCM4-7_v0_day_19700101-20051231.nc,cordex,output,AUS-44i,ICTP,r1i1p1,ICTP-RegCM4-7,v0,day,tasmin,v19920121,19700101-20051231.nc
2,CCLM5-0-15,MOHC-HadGEM2-ES,historical,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/tasmin_AUS-44i_MOHC-HadGEM2-ES_historical_r1i1p1_CLMcom-HZG-CCLM5-0-15_v1_day_19500101-20051230.nc,cordex,output,AUS-44i,CLMcom-HZG,r1i1p1,CLMcom-HZG-CCLM5-0-15,v1,day,tasmin,v19920121,19500101-20051230.nc
3,CCAM-1704,CNRM-CERFACS-CNRM-CM5,historical,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/tasmin_AUS-44i_CNRM-CERFACS-CNRM-CM5_historical_r1i1p1_CSIRO-CCAM-1704_v1_day_19600101-20051231.nc,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-1704,v1,day,tasmin,v19920121,19600101-20051231.nc
4,CCAM-1704,MIROC-MIROC5,historical,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/historical/pr_AUS-44i_MIROC-MIROC5_historical_r1i1p1_CSIRO-CCAM-1704_v1_day_19600101-20051231.nc,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-1704,v1,day,pr,v19920121,19600101-20051231.nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,WRF360J,CSIRO-BOM-ACCESS1-3,rcp45,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/rcp45/pr_AUS-44i_CSIRO-BOM-ACCESS1-3_rcp45_r1i1p1_UNSW-WRF360J_v1_day_20060101-21001231.nc,cordex,output,AUS-44i,UNSW,r1i1p1,UNSW-WRF360J,v1,day,pr,v19920121,20060101-21001231.nc
203,CCAM-1704,MOHC-HadGEM2-CC,rcp45,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/rcp45/tasmax_AUS-44i_MOHC-HadGEM2-CC_rcp45_r1i1p1_CSIRO-CCAM-1704_v1_day_20060101-20991231.nc,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-1704,v1,day,tasmax,v19920121,20060101-20991231.nc
204,WRF360K,CCCma-CanESM2,rcp45,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/rcp45/tasmin_AUS-44i_CCCma-CanESM2_rcp45_r1i1p1_UNSW-WRF360K_v1_day_20060101-20991231.nc,cordex,output,AUS-44i,UNSW,r1i1p1,UNSW-WRF360K,v1,day,tasmin,v19920121,20060101-20991231.nc
205,CCAM-2008,NOAA-GFDL-GFDL-ESM2M,rcp45,/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/rcp45/pr_AUS-44i_NOAA-GFDL-GFDL-ESM2M_rcp45_r1i1p1_CSIRO-CCAM-2008_v1_day_20060101-20991231.nc,cordex,output,AUS-44i,CSIRO,r1i1p1,CSIRO-CCAM-2008,v1,day,pr,v19920121,20060101-20991231.nc


In [10]:
# Write new catalogue to a csv file
new_catalogue.to_csv('/g/data/w40/ri9247/josh/catalogue.csv',index = False)

## Step 4 - Troubleshooting
Here, we troubleshoot a few simulations with unique pre-processing issues including:
- subsetting historical and RCP85 WRF pr files to match the latitude and longitude 
- correcting CCAM-2008 calendar mismatch

### Subsetting WRF pr files

In [6]:
# Define list with experiments
experiments = [
    'historical',
    'rcp85'
]

# Define master path of input datasets
input_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/tmp_WRF_pr_files/'
output_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/'

In [7]:
for experiment in experiments:
        
    # Iterate through the files in the given directory
    for filename in os.listdir(input_master_path + experiment):
            
        print("Starting: ", filename)
        
        # Use CDO to subset lat and lon boundaries
        !cdo -sellonlatbox,89.75,205.75,9.75,-51.75 {input_master_path + experiment + '/' + filename} {output_master_path + experiment + '/' + filename}

Starting:  pr_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19510101-20051231.nc
cdo    sellonlatbox: Processed 635897206 values from 1 variable over 20089 timesteps [42.01s 94MB].
Starting:  pr_AUS-44i_CSIRO-BOM-ACCESS1-3_historical_r1i1p1_UNSW-WRF360J_v1_day_19510101-20051231.nc
cdo    sellonlatbox: Processed 635897206 values from 1 variable over 20089 timesteps [46.27s 92MB].
Starting:  pr_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360K_v1_day_19510101-20051231.nc
cdo    sellonlatbox: Processed 635897206 values from 1 variable over 20089 timesteps [45.10s 96MB].
Starting:  pr_AUS-44i_CSIRO-BOM-ACCESS1-3_historical_r1i1p1_UNSW-WRF360K_v1_day_19510101-20051231.nc
cdo    sellonlatbox: Processed 635897206 values from 1 variable over 20089 timesteps [51.09s 98MB].
Starting:  pr_AUS-44i_CCCma-CanESM2_rcp85_r1i1p1_UNSW-WRF360J_v1_day_20060101-20991231.nc
cdo    sellonlatbox: Processed 1086776782 values from 1 variable over 34333 timesteps [111.76s 106MB].
Sta

### Correcting the calendar for CCAM-2008 files

In [4]:
# Define list with experiments
experiments = [
    'historical',
    'rcp45',
    'rcp85'
]

# Define master path of input datasets
input_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/tmp_CCAM_calendar/'
output_master_path = '/scratch/w40/ri9247/tmp/cordex_preproc/step2_regrid_AUS-44i/'

In [ ]:
for experiment in experiments:
        
    # Iterate through the files in the given directory
    for filename in os.listdir(input_master_path + experiment):
        
        print("Starting: ", filename)
        
        # Use CDO to update the calendar
        # Make the unit of equal time
        !cdo -settaxis,1900-01-01,00:00:00,1day {input_master_path + experiment + '/' + filename} {'temp_cal_step1.nc'}
        
        # Make all files share the 365-day calendar
        !cdo -setcalendar,365_day {'temp_cal_step1.nc'} {output_master_path + experiment + '/' + filename}

Starting:  tasmin_AUS-44i_MIROC-MIROC5_historical_r1i1p1_CSIRO-CCAM-2008_v1_day_19610101-20051231.nc
cdo    settaxis (Warning): Time bounds unsupported by this operator, removed!
cdo    settaxis: Processed 519916950 values from 1 variable over 16425 timesteps [36.78s 86MB].
cdo    setcalendar: Processed 519916950 values from 1 variable over 16425 timesteps [11.21s 83MB].
Starting:  tasmin_AUS-44i_NCC-NorESM1-M_historical_r1i1p1_CSIRO-CCAM-2008_v1_day_19600101-20051231.nc
cdo    settaxis (Warning): Time bounds unsupported by this operator, removed!
cdo    settaxis: Processed 531470660 values from 1 variable over 16790 timesteps [75.78s 86MB].
cdo    setcalendar: Processed 531470660 values from 1 variable over 16790 timesteps [11.39s 83MB].
Starting:  pr_AUS-44i_NCC-NorESM1-M_historical_r1i1p1_CSIRO-CCAM-2008_v1_day_19600101-20051231.nc
cdo    settaxis: Processed 531504240 values from 2 variables over 16790 timesteps [14.17s 92MB].
cdo    setcalendar: Processed 531504240 values from 2 va